In [ ]:
# Loading an Image
import cv2

# Load an image from file
image = cv2.imread(r'H:\01_Training\Batch51\05_DeepLearning\OpenCV\zebra.jpg')

# Display the image
cv2.imshow('Image', image)
key = cv2.waitKey(0)  # Wait for a key press
cv2.destroyAllWindows() # close active image windows
print(key)


119


In [6]:
import cv2

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start the webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('Face Detection', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


In [9]:
import cv2
import numpy as np

# Start the webcam
cap = cv2.VideoCapture(1)

# Define the color range to track (HSV format for blue color) HSV = Hue, Saturation, and Value
lower_blue = np.array([100, 150, 0])
upper_blue = np.array([140, 255, 255])

# Define the color range to track (HSV format for green color)
# lower_green = np.array([35, 150, 50])    # Lower bound for green
# upper_green = np.array([85, 255, 255])   # Upper bound for green


while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create a mask for blue color
    # Pixels that fall within the specified color range are set to 255 (white).
    # Pixels that fall outside the color range are set to 0 (black).
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Apply the mask to the original frame
    # The areas matching the target color (blue) will remain as they are in the original image (frame).
    # The areas not matching the target color will be set to black (because the corresponding pixels in the mask are 0).
    result = cv2.bitwise_and(frame, frame, mask=mask)

    # Display the frames
    cv2.imshow('Original Frame', frame)
    cv2.imshow('Mask', mask)
    cv2.imshow('Tracked Color', result)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


In [8]:
import cv2
import numpy as np

# Load YOLO model
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

# Load the COCO dataset class labels
with open('coco.names', 'r') as f:
    classes = f.read().strip().split('\n')

# Load an image
image = cv2.imread('zebra+and+elephant.png')
height, width = image.shape[:2]

# Convert the image to a blob
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)

# Get output layer names
output_layers = net.getUnconnectedOutLayersNames()

# Forward pass
detections = net.forward(output_layers)

# Draw bounding boxes
for detection in detections:
    for obj in detection:
        scores = obj[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > 0.5:
            # Object detected
            center_x, center_y, w, h = (obj[0:4] * np.array([width, height, width, height])).astype('int')

            # Get top-left corner
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            # Draw rectangle and label
            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(image, f'{classes[class_id]}: {confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the image
cv2.imshow('YOLO Object Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [7]:
import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")  # Load the YOLO model
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load class labels (COCO dataset classes)
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Set up video capture (using the webcam)
cap = cv2.VideoCapture(1)  # Use '0' for webcam, or provide a video path for a file

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break
    
    # Resize the frame to fit the model (YOLO expects 416x416 size)
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    # Feed the blob into the YOLO network
    net.setInput(blob)
    outputs = net.forward(output_layers)

    # Process detections
    class_ids = []
    confidences = []
    boxes = []

    for output in outputs:
        for detection in output:
            scores = detection[5:]  # Get the scores (confidence) for each class
            class_id = np.argmax(scores)  # Class with the highest confidence score
            confidence = scores[class_id]
            
            if confidence > 0.5:  # If confidence is above threshold
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Draw rectangle around the detected object
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Non-maxima suppression to remove redundant overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.4)

    # Draw bounding boxes and labels on the frame
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} {confidence:.2f}", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame with detected objects
    cv2.imshow("Real-Time Object Detection (YOLO)", frame)

    # Break loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
